<a href="https://colab.research.google.com/github/lawielas/Hamoye-Data-Science-Internship/blob/master/Hamoye3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np

In [2]:
#loading dataset from github
url='https://raw.githubusercontent.com/lawielas/Hamoye-Data-Science-Internship/master/Data_for_UCI_named.csv'

In [3]:
grid_system= pd.read_csv(url, error_bad_lines=False)

In [4]:
grid_system.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [5]:
grid_system.describe(include='all')

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unstable
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6380
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731,NaN
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919,NaN
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760,NaN
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557,NaN
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142,NaN
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878,NaN


In [7]:
grid_system.drop(columns=[ 'stab' ], inplace = True)

KeyError: ignored

In [8]:
grid_system[ 'stabf' ].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [9]:
grid_system.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stabf    0
dtype: int64

In [10]:
X = grid_system.drop(columns= 'stabf' )
Y = grid_system[ 'stabf' ]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2 , random_state= 1 )
Y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

In [12]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state= 1 )
X_train_balanced, Y_balanced = smote.fit_sample(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [14]:
normalised_train_df = scaler.fit_transform(X_train_balanced)
normalised_train_df = pd.DataFrame(normalised_train_df)


normalised_test_df = scaler.transform(X_test)
normalised_test_df = pd.DataFrame(normalised_test_df)

In [15]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=1)
log_reg.fit(normalised_train_df,Y_balanced)
#returns
LogisticRegression(C= 1.0 , class_weight= None , dual= False , fit_intercept= True ,
intercept_scaling= 1 , l1_ratio= None , max_iter= 100 ,
multi_class= 'auto' , n_jobs= None , penalty= 'l2' ,
random_state= None , solver= 'lbfgs' , tol= 0.0001 , verbose= 0 ,
warm_start= False )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, Y_balanced, cv= 5 , scoring= 'f1_macro' )
scores

array([0.80215317, 0.81786784, 0.80510386, 0.83356275, 0.80421639])

In [17]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=Y_test, y_pred=new_predictions, labels=[ 'unstable' , 'stable' ])
cnf_mat

array([[1019,  269],
       [ 160,  552]])

In [18]:
accuracy = accuracy_score(y_true=Y_test, y_pred=new_predictions)
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 ))

Accuracy: 79.0


In [19]:
precision = precision_score(y_true=Y_test, y_pred=new_predictions, pos_label= 'unstable' )

print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 86.0


In [20]:
recall = recall_score(y_true=Y_test, y_pred=new_predictions, pos_label= 'unstable' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 79.0


In [21]:
f1 = f1_score(y_true=Y_test, y_pred=new_predictions, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 83.0


In [22]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, Y_balanced)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [23]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state= 1)
clf.fit(normalised_train_df, Y_balanced)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [24]:
from sklearn.model_selection import cross_val_score
scores2 = cross_val_score(clf, normalised_train_df, Y_balanced, cv= 5 , scoring= 'f1_macro' )
scores2

array([0.92439861, 0.92979764, 0.92095603, 0.95919397, 0.95374065])

In [25]:
from sklearn.model_selection import KFold
kf = KFold(n_splits= 5 )
kf.split(normalised_train_df)


for train_index, test_index in kf.split(normalised_train_df):
    X_train, X_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    y_train, y_test = Y_balanced[train_index], Y_balanced[test_index]
    clf_tree=clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print((f1_score(y_true=y_test, y_pred=clf.predict(X_test),
    pos_label= 'unstable' )* 100))


92.9678188319428
93.57652656621728
93.73493975903615
93.27731092436974
0.0


In [26]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1 )
#run for every split
for train_index, test_index in skf.split(normalised_train_df, Y_balanced):
    X_train, X_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    y_train, y_test = Y_balanced[train_index], Y_balanced[test_index]
    model = clf.fit(X_train, y_train)
    print((f1_score(y_true=y_test, y_pred=model.predict(X_test), pos_label= 'unstable' )))


0.9401794616151546
0.9386934673366835
0.927752864972596
0.9425287356321839
0.926463231615808


In [27]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions2 = clf.predict(normalised_test_df)
cnf_mat2 = confusion_matrix(y_true=Y_test, y_pred=new_predictions, labels=[ 'unstable' , 'stable' ])
cnf_mat2

array([[1019,  269],
       [ 160,  552]])

In [61]:
accuracy2 = accuracy_score(y_true=Y_test, y_pred=new_predictions2)
print(accuracy2)

0.912


In [29]:
precision = precision_score(y_true=Y_test, y_pred=new_predictions2, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 95.0


In [30]:
recall = recall_score(y_true=Y_test, y_pred=new_predictions2, pos_label= 'unstable' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 92.0


In [31]:
f1 = f1_score(y_true=Y_test, y_pred=new_predictions2, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 93.0


In [32]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(random_state=1)
et.fit(normalised_train_df, Y_balanced)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [33]:
from sklearn.model_selection import cross_val_score
scores3 = cross_val_score(et, normalised_train_df, Y_balanced, cv= 5 , scoring= 'f1_macro' )
scores3

array([0.92782503, 0.94403048, 0.94157715, 0.96165642, 0.96607105])

In [34]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions3 = et.predict(normalised_test_df)
cnf_mat3 = confusion_matrix(y_true=Y_test, y_pred=new_predictions, labels=[ 'unstable' , 'stable' ])
cnf_mat3

array([[1019,  269],
       [ 160,  552]])

In [35]:
accuracy = accuracy_score(y_true=Y_test, y_pred=new_predictions3)
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 ))

Accuracy: 92.0


In [36]:
precision = precision_score(y_true=Y_test, y_pred=new_predictions3, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 94.0


In [37]:
recall = recall_score(y_true=Y_test, y_pred=new_predictions3, pos_label= 'unstable' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 93.0


In [38]:
f1 = f1_score(y_true=Y_test, y_pred=new_predictions3, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 94.0


In [39]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost.sklearn import XGBRegressor
XGBC = XGBClassifier(random_state=1)
XGBC.fit(normalised_train_df, Y_balanced)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [40]:
from sklearn.model_selection import cross_val_score
scores4 = cross_val_score(XGBC, normalised_train_df, Y_balanced, cv= 5 , scoring= 'f1_macro' )
scores4

array([0.90770098, 0.91948384, 0.91456937, 0.94545426, 0.9380671 ])

In [69]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions4 = XGBC.predict(normalised_test_df)
cnf_mat4 = confusion_matrix(y_true=Y_test, y_pred=new_predictions4, labels=[ 'unstable' , 'stable' ])
cnf_mat4

array([[1174,  114],
       [  65,  647]])

In [70]:
accuracy1 = accuracy_score(y_true=Y_test, y_pred=new_predictions4)
print( accuracy1)

0.9105


In [43]:
precision = precision_score(y_true=Y_test, y_pred=new_predictions4, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 95.0


In [44]:
recall = recall_score(y_true=Y_test, y_pred=new_predictions4, pos_label= 'unstable' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 91.0


In [45]:
f1 = f1_score(y_true=Y_test, y_pred=new_predictions4, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 93.0


In [46]:
import lightgbm as lgbm
light=lgbm.LGBMClassifier(random_state=1)
light.fit(normalised_train_df, Y_balanced)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [47]:
from sklearn.model_selection import cross_val_score
scores5 = cross_val_score(light, normalised_train_df, Y_balanced, cv= 5 , scoring= 'f1_macro' )
scores5

array([0.94010799, 0.95090478, 0.94893802, 0.97150747, 0.96805461])

In [48]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions5 = light.predict(normalised_test_df)
cnf_mat5 = confusion_matrix(y_true=Y_test, y_pred=new_predictions, labels=[ 'unstable' , 'stable' ])
cnf_mat5

array([[1019,  269],
       [ 160,  552]])

In [62]:
accuracy3 = accuracy_score(y_true=Y_test, y_pred=new_predictions5)
print(accuracy3)

0.9345


In [50]:
precision = precision_score(y_true=Y_test, y_pred=new_predictions5, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 96.0


In [51]:
recall = recall_score(y_true=Y_test, y_pred=new_predictions5, pos_label= 'unstable' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 94.0


In [52]:
f1 = f1_score(y_true=Y_test, y_pred=new_predictions5, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 95.0


In [53]:
from sklearn.ensemble import ExtraTreesClassifier
et2 = ExtraTreesClassifier(random_state=1)
rf_params = {'n_estimators' : [50, 100, 300, 500, 1000],
             'min_samples_split' : [2, 3, 5, 7, 9],
             'min_samples_leaf' : [1, 2, 4, 6, 8],
             'max_features' : ['auto', 'sqrt', 'log2', None]}
n_iter=10

In [68]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(et2, 
                                   cv=5,
                                   verbose=1, 
                                   param_distributions=rf_params,
                                   n_iter=n_iter, 
                                   n_jobs=-1, 
                                   scoring='accuracy',
                                   random_state = 1)
random_search.fit(X_train, y_train)
print('Best params={}'.format(random_search.best_params_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


Best params={'n_estimators': 300, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'auto'}
